In [70]:
import pandas as pd
from reco_systems.CF_knn import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
folder = "database_cleaned"
avis_clean = pd.read_csv(f"database_cleaned/avis_clean.csv", index_col=0)
jeux_clean = pd.read_csv(f"database_cleaned/jeux_clean.csv", index_col=0)
#users = pd.read_csv(f"trictrac_database/users.csv", names=["Username", "User id"])
#users.shape

In [3]:
rev_filter, table_assoc_games= filter_games(avis_clean, min_reviews=10) 
#table_assoc.values, np.unique(rev_filter["User id"])
np.unique(rev_filter["User id"])
rev_filter[(rev_filter["Username"] == "limp") & (~rev_filter["Rating"].isna())].shape

(795, 8)

In [14]:
matrix_ratings, mask_matrix = get_matrix_user_game(rev_filter)
matrix_ratings.toarray()

array([[0. , 0. , 8.6, ..., 0. , 0. , 0. ],
       [0. , 0. , 6.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 6. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], shape=(11201, 2888))

In [ ]:
similarity_matrix = calc_similarity_matrix(matrix_ratings, dist_type="cos")
similarity_matrix.shape

(11201, 11201)

In [20]:
# Print firs 10 users (10 rows, 10 columns)
similarity_matrix[:10, :10]

array([[0.        , 0.9156413 , 0.82997579, 0.93598729, 0.96397752,
        0.92237635, 0.98039342, 0.9745858 , 0.98498027, 1.        ],
       [0.9156413 , 0.        , 0.70390267, 0.6720784 , 0.89248414,
        0.93898508, 0.88905439, 0.84010393, 0.83077079, 0.9533165 ],
       [0.82997579, 0.70390267, 0.        , 0.81100859, 0.94595554,
        0.90644913, 0.9330459 , 0.9097209 , 0.92788905, 0.95298363],
       [0.93598729, 0.6720784 , 0.81100859, 0.        , 0.93925896,
        0.97102855, 0.95790982, 0.83950071, 0.91542438, 1.        ],
       [0.96397752, 0.89248414, 0.94595554, 0.93925896, 0.        ,
        0.86858314, 0.96680605, 0.98021069, 1.        , 1.        ],
       [0.92237635, 0.93898508, 0.90644913, 0.97102855, 0.86858314,
        0.        , 0.92847145, 1.        , 1.        , 1.        ],
       [0.98039342, 0.88905439, 0.9330459 , 0.95790982, 0.96680605,
        0.92847145, 0.        , 0.98536335, 0.94273551, 1.        ],
       [0.9745858 , 0.84010393, 0.9097209

In [68]:
similar_users = get_KNN(similarity_matrix, k=30, user_ind=3000)
similar_users

array([2548, 4930, 3066, 2994, 1941, 1908, 1906, 1815,  201, 2250, 2275,
         95,   94, 1192,  384, 2934,  580,  110, 2589, 3160, 2272, 1724,
       8883, 2245, 1199, 2180, 1904, 2430, 2590, 2936])

In [293]:
for user in similar_users:
    print(user, np.count_nonzero(matrix_ratings.toarray()[user]))

1815 233
6794 6
7071 4
4026 3
1930 23
6150 6
7727 4
1975 11
10396 3
1939 215
812 49
6811 2
10320 2
3443 7
2538 117
4998 3
4711 4
2991 4
1938 172
6626 2
5019 13
5710 8
4618 7
1229 6
1906 285
8018 13
2608 5
2890 92
1050 53
2290 30


In [71]:
ratings = predict_ratings_baseline(matrix_ratings, mask_matrix, similar_users)

In [81]:
games_ind = get_games_ind(ratings, n=500)
games_ind

array([1328,  778, 2542,  784,  786,  793,    6, 1987, 2552,  794,  803,
        804, 1972,  762, 2557, 1968,  759,  758, 2532, 2527, 1962,  823,
        827,  751, 2564, 2021,  829,  835,  837, 2024,  840, 2866, 1937,
       2864,   34, 2519,  842,  843, 2571,  845,   40,  848, 2031, 1933,
         44, 2512,  870, 2506,  877, 2040,  884, 1918,  885, 1909,  889,
       1901,  891, 1897,  900,  904,  910, 1890, 2490, 1888, 2053,  714,
        919, 2054, 1885, 2584,  710,   71,   72,  709,   74, 2057,  708,
       2058,  707,  706,   80,  704,  702,  701, 2836, 1870,  934, 2835,
         88,   89,   90, 2834, 2477,  941,  944,   95, 1868, 2066,  694,
         99,  945,  946,  947, 2592,  954, 2072, 1848, 2074, 1837, 2083,
       2084, 2467,  684, 1830, 2465,  964, 1825,  968,  118,  969, 1815,
        677,  676, 2459,  674, 1801,  973, 2457, 2109,  975, 2456, 1768,
       2596,  980, 2451, 2813,  664,  137, 2445,  139, 2598, 1757, 2599,
       1753, 2809, 2601, 1000, 2441,  148,  149, 10

In [82]:
df_pred_games = get_games_df(jeux_clean, table_assoc_games, games_ind)

In [83]:
avis_clean[["User id", "Game id", "Game name UI", "Rating"]][(avis_clean["User id"] == 3000)].merge(df_pred_games["Game id"], on="Game id")

,User id,Game id,Game name UI,Rating
0,3000,2921,El Grande - Edicion 20e Aniversario,6.0
1,3000,9995,War of the ring : Anniversary Release,10.0
2,3000,8092,Saint Pétersbourg,10.0
3,3000,6269,Medina,4.0
4,3000,9736,Unanimo,10.0
5,3000,3577,Glastonbury,10.0
6,3000,9316,Tigris & Euphrates,10.0
7,3000,9890,Vikings,10.0
8,3000,3959,Herr der Ringe: Die Entscheidung,4.0
9,3000,8275,Serenissima,10.0
